# DLCD itteration comparison tool 

This notebook is designed to detect change between different itterations of the DLCD, 
using as little compute power as posible.

as a result, this notebook dosn't have any images, however they can be easily generated from the xarrays found within. To run this notebook you will need acess to the DLCD itterations in the form of geotifs. The out put is a confusion matrix in CSV file format, see the end of the notebook to set the location that the CSV file is saved. 

In the output CSV file the landcover types for the old itteration are along the X axis, (across the top of the file in the first row) the landcover types for the new itteration run down the Y axis (in the first collum of the file) ie:

    old old old old old old old ...
new 100   0   0   2  31   0  12 ...
new   0 200   0   0   1   1   0 ...
new   1   0 150   1   4   0   0 ...
...




In [1]:
'''
landcover classification legend

number 	landcover type 
0	no data
1	Extraction Sites
3	Inland Waterbodies
4	Salt Lakes
5	Irrigated Cropping
6	Irrigated Pasture
7	Irrigated Sugar
8	Rainfed Cropping
9	Rainfed Pasutre
10	Rainfed Sugar
11	Wetlands
14	Tussock Grasses - Closed
15	Alpine Grasses - Open
16	Hummock Grasses - Open
18	Tussock Grasses - Open
19	Shrubs and Grasses - Sparse-Scattered
24	Shrubs - Closed
25	Shrubs - Open
31	Trees - Closed
32	Trees - Open
33	Trees - Scattered
34	Trees - Sparse
35	Built up area '''


'\nlandcover classification legend\n\nnumber \tlandcover type \n0\tno data\n1\tExtraction Sites\n3\tInland Waterbodies\n4\tSalt Lakes\n5\tIrrigated Cropping\n6\tIrrigated Pasture\n7\tIrrigated Sugar\n8\tRainfed Cropping\n9\tRainfed Pasutre\n10\tRainfed Sugar\n11\tWetlands\n14\tTussock Grasses - Closed\n15\tAlpine Grasses - Open\n16\tHummock Grasses - Open\n18\tTussock Grasses - Open\n19\tShrubs and Grasses - Sparse-Scattered\n24\tShrubs - Closed\n25\tShrubs - Open\n31\tTrees - Closed\n32\tTrees - Open\n33\tTrees - Scattered\n34\tTrees - Sparse\n35\tBuilt up area '

In [2]:
#load requiered modules 
import numpy as np
import rasterio
#from matplotlib import pyplot as plt #don't need unless want to plot
import xarray as xr
import pandas as pd

In [3]:
#load in geotifs using rasterio, trun into an xarray :
#enter filename as a string, include subfolders!!
#previous_version DLCD found in old_DLCD, new in new_DLCD.

#use this command to print a list of all file in respective folder : 

def load_gfif_full(file_name, x):
    #file_string = 'old_DLCD/'+file_name+'.tif'

    dataset = rasterio.open(file_name) #open using Raterio
    {i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}
    data_array = dataset.read(x) #read selected band into np array
    data_xr = xr.DataArray(data_array) #turn into Xr array for fuctionality #Now don't need xr yet
    return(data_xr) #return np array into program 
    


old_version = load_gfif('old_DLCD/DLCDv2_2002_20031.tif',1)#for old dlcd files all only have ONE band (are single year)

new_version = load_gfif('new_DLCD/DLCDv2_2016_new.tif',2)#for new DLCD band number corrosponds to year in following table

band number  year:

1   = 2001-2002

2   = 2002-2003

3   = 2003-2004

ect...

15  = 2015-2016


In [4]:
#this is potential usefull in the future, but not needed here
'''
# define function to return the number of pixles occupied by the selected landcover type. 

def count_type(in_xr, X):
    raw_np = ones_like(old_np)
    raw_xr = xr.DataArray(raw_np)
    count_name = raw_xr.where(in_xr==X) #makes a xarray with just the selected class an nan values
    count_name = count_name.fillna(0) #replaces all the nan with 0
    howmuch_name = np.count_nonzero(count_name) #counts the number of non 0 in the xaray
    return(howmuch_name) #returns an int which is the number of pixles that exist of the specified LC class 
    '''

In [5]:
# define function to return the number of pixles which are class A in old and class B in new: 

def count_overlap(old_np,new_np,A,B):
    raw_np = np.ones_like(old_np) #generate a np with the same size as the dlcd but full of 1 values
    raw_xr = xr.DataArray(raw_np) #turn to xarray for it's useful fuctionality
    count_old = raw_xr.where(old_np==A) # make an xr where 1= presance of class A, 0= absence 
    count_new = count_old.where(new_np==B) # make an xr where 1= overlap of classes A&B
    count_class = count_new.fillna(0) #replaces all the nan with 0
    howmuch_name = np.count_nonzero(count_class) #counts the number of non 0 in the xaray


    return(howmuch_name) #returns an int which is the number of pixles that are A in old and B in new 

In [6]:
#create a list of class values (1-35) to compair in old v new. 

class_values = [
1,
3,
4,
5,
6,
7,
8,
9,
10,
11,
14,
15,
16,
18,
19,    
24,
25,
31,
32,
33,
34,
35]

'\n7,\n8,\n9,\n10,\n11,\n14,\n15,\n16,\n18,\n19,\n22,    \n24,\n25,\n31,\n32,\n33,\n34,\n35\n]'

In [7]:
#define a dictionary to enter all of the comparison scores in. this will become a dict or nested dicts, 
#the structure will be like this:
#{class 1: {class 1 : 44765, class 3: 0, class 4: 0, class 5: 0, class 6: 0, ...ect},
#class 3: {class 1: 0, class 3: 426732, class 4: 1771, class 5: 0, class 6: 0, ...ect},
#class 4: {class 1: 0, class 3: 961, class 4: 1450161, class 5: 0, class 6: 0, ...ect},
#ect...
#where each sub-dict records overlap values of a single class in the old DLCD against each class in the new DLCD

matrix_d = dict()

#create loop to populate matrix dict
for i in class_values:
    i_lable = int(i) #class names have to be integer so pandas will sort into numerical order
    d = dict() #mini dict to be nested in larger
    for x in class_values:
        x_lable =int(x)
        change = count_overlap(old_version,new_version,i,x) #use fuction defined above
        d[x_lable] = change
        #print(i,x,change)
    matrix_d[i_lable] = d #add nested dicts to master dict
    #print('added')

matrix_dataframe = pd.DataFrame(matrix_d) #turn large dict to pandas dataframe 

In [8]:
#this dataframe will become the CSV file.

matrix_dataframe

{1: {1: 44765, 3: 0, 4: 0, 5: 0, 6: 0},
 3: {1: 0, 3: 426732, 4: 1771, 5: 0, 6: 0},
 4: {1: 0, 3: 961, 4: 1450161, 5: 0, 6: 0},
 5: {1: 0, 3: 0, 4: 0, 5: 308603, 6: 79},
 6: {1: 0, 3: 0, 4: 0, 5: 74, 6: 42964}}

In [10]:
matrix_dataframe.to_csv('Matrixes/confustion_matrix_for_2002.csv')#save the panda dataframe to a CSV file